In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.api import OLS

In [35]:
df_housing = pd.read_excel("hackathon/Housing.xlsx", usecols=["Area", "Property: Owner-occupied"], index_col="Area")
df_income = pd.read_excel("hackathon/Income.xlsx", usecols=["Area", "Mean disposable household income"], index_col="Area")
df_population = pd.read_excel("hackathon/Population.xlsx", usecols=["Area", "Population density"], index_col="Area")
df_solar = pd.read_excel("hackathon/Solar panels.xlsx", usecols=["Area", "Year", "nl_wp_Wonen"], index_col="Area")

In [44]:
df_solar = df_solar.loc[df_solar["Year"] == 2021]
df_solar = df_solar.drop(columns=["Year"])

In [45]:
pd.concat([df_housing, df_income, df_population, df_solar], axis=1)

,Property: Owner-occupied,Mean disposable household income,Population density,nl_wp_Wonen
Area,,,,
Haarlemmerbuurt,1556,49800,17126,252054.0
Jordaan,3236,46700,23298,276128.0
Grachtengordel-West,1656,83300,14217,75086.0
Burgwallen-Nieuwe Zijde,522,44300,7182,45236.0
Burgwallen-Oude Zijde,526,40800,12986,14979.0
...,...,...,...,...
K-buurt,846,37000,8248,904655.0
Holendrecht,822,35000,7220,1038394.0
Nellestein,679,38600,1418,166320.0
